# Graphs

A **graph** is a mathematical structure used to model pairwise relationships between entities. A graph consists of:

- **Nodes**: The objects
- **Edges**: The connections between objects, which can be directed (arrows) or undirected


### `__init__(self, g = {})`

Initialises the graph. The input `g` is a dictionary in the format:

```python
{
    "A": [("B", 3), ("C", 2)],
    "B": [("C", 1)]
}
```
## Basic Graph Operations

### `add_vertex(self, v)`
Adds a vertex `v` to the graph if it doesn’t exist.

### `add_edge(self, o, d, w)`
Adds an edge from origin `o` to destination `d` with weight `w`.

---

## Graph Inspection Methods

### `get_nodes(self)`
Returns a list of all nodes in the graph.

### `get_edges(self)`
Returns all edges in the graph as a list of `(origin, destination, weight)`.

### `print_graph(self)`
Prints the adjacency list: each node and its outgoing connections.

### `size(self)`
Returns a tuple: `(number of vertices, number of edges)`.

---

## Neighbourhood Functions

### `get_successors(self, v)`
Returns the list of nodes directly reachable from `v` (outgoing edges).

### `get_predecessors(self, v)`
Returns the list of nodes that point to `v` (incoming edges).

### `get_adjacents(self, v)`
Returns all nodes adjacent to `v` (successors + predecessors).

---

## Degree Calculations

### `out_degree(self, v)`
Returns the number of outgoing edges from `v`.

### `in_degree(self, v)`
Returns the number of incoming edges to `v`.

### `degree(self, v)`
Returns the total degree (in + out) of node `v`.

---

## Path & Distance

### `distance(self, s, d)`
Returns the shortest path distance from node `s` to `d` using Dijkstra’s algorithm.

### `shortest_path(self, s, d)`
Returns the actual shortest path as a list of nodes from `s` to `d`.

### `_dijkstra(self, source)`
Internal method implementing Dijkstra’s algorithm:

- Returns two dictionaries:
  - `dist[node]`: Shortest distance from source to node
  - `prev[node]`: Previous node in the shortest path

---

## Reachability

### `reachable_bfs(self, v)`
Returns all nodes reachable from `v` using Breadth-First Search(BFS).

### `reachable_dfs(self, v)`
Returns all nodes reachable from `v` using Depth-First Search(DFS).

### `reachable_with_dist(self, s)`
Returns a list of tuples `(node, distance)` representing all reachable nodes from `s` and their respective distances (using BFS-style levels).

---

## Cycle Detection

### `node_has_cycle(self, v)`
Checks whether a cycle exists starting and ending at node `v`.

### `has_cycle(self)`
Checks the entire graph for the presence of any cycle.

In [3]:
class MyGraph:

    def __init__(self, g = {}):
        self.graph = g

    def print_graph(self):
        for v in self.graph:
            print(v, "->", self.graph[v])

    def get_nodes(self):
        return list(self.graph.keys())

    def get_edges(self):
        edges = []
        for v in self.graph:
            for d, w in self.graph[v]:
                edges.append((v, d, w))
        return edges

    def size(self):
        return len(self.get_nodes()), len(self.get_edges())

    def add_vertex(self, v):
        if v not in self.graph:
            self.graph[v] = []

    def add_edge(self, o, d, w):
        if o not in self.graph:
            self.add_vertex(o)
        if d not in self.graph:
            self.add_vertex(d)
        self.graph[o].append((d, w))

    def get_successors(self, v):
        return [dest for dest, _ in self.graph[v]]

    def get_predecessors(self, v):
        preds = []
        for node in self.graph:
            for dest, _ in self.graph[node]:
                if dest == v:
                    preds.append(node)
        return preds

    def get_adjacents(self, v):
        return list(set(self.get_successors(v) + self.get_predecessors(v)))

    def out_degree(self, v):
        return len(self.graph[v])

    def in_degree(self, v):
        return len(self.get_predecessors(v))

    def degree(self, v):
        return self.in_degree(v) + self.out_degree(v)

    def distance(self, s, d):
        if s == d: return 0
        dist, _ = self._dijkstra(s)
        return dist.get(d, None)

    def shortest_path(self, s, d):
        if s == d: return [s]
        dist, prev = self._dijkstra(s)
        if d not in dist:
            return None
        path = []
        current = d
        while current != s:
            path.append(current)
            current = prev.get(current)
            if current is None:
                return None
        path.append(s)
        path.reverse()
        return path

    def _dijkstra(self, source):
        unvisited = {node: float('inf') for node in self.graph}
        unvisited[source] = 0
        prev = {}
        visited = {}

        while unvisited:
            u = min(unvisited, key=unvisited.get)
            current_dist = unvisited[u]
            visited[u] = current_dist
            del unvisited[u]

            for v, weight in self.graph[u]:
                if v in visited:
                    continue
                new_dist = current_dist + weight
                if new_dist < unvisited.get(v, float('inf')):
                    unvisited[v] = new_dist
                    prev[v] = u

        return visited, prev

    def reachable_bfs(self, v):
        l = [v]
        res = []
        while l:
            node = l.pop(0)
            if node != v: res.append(node)
            for elem, _ in self.graph[node]:
                if elem not in res and elem not in l and elem != node:
                    l.append(elem)
        return res

    def reachable_dfs(self, v):
        l = [v]
        res = []
        while l:
            node = l.pop(0)
            if node != v: res.append(node)
            s = 0
            for elem, _ in self.graph[node]:
                if elem not in res and elem not in l:
                    l.insert(s, elem)
                    s += 1
        return res

    def reachable_with_dist(self, s):
        res = []
        l = [(s, 0)]
        while l:
            node, dist = l.pop(0)
            if node != s:
                res.append((node, dist))
            for elem, _ in self.graph[node]:
                if not is_in_tuple_list(l, elem) and not is_in_tuple_list(res, elem):
                    l.append((elem, dist + 1))
        return res

    def node_has_cycle(self, v):
        l = [v]
        visited = [v]
        while l:
            node = l.pop(0)
            for elem, _ in self.graph[node]:
                if elem == v:
                    return True
                elif elem not in visited:
                    l.append(elem)
                    visited.append(elem)
        return False

    def has_cycle(self):
        for v in self.graph:
            if self.node_has_cycle(v):
                return True
        return False

def is_in_tuple_list(tl, val):
    for x, _ in tl:
        if val == x:
            return True
    return False

def test_graph():
    g = {
        1: [(2, 3), (3, 1)],
        2: [(3, 7), (4, 5)],
        3: [(4, 2)],
        4: []
    }
    wg = MyGraph(g)
    wg.print_graph()
    print("Nodes:", wg.get_nodes())
    print("Edges:", wg.get_edges())
    print("Shortest path 1->4:", wg.shortest_path(1, 4))
    print("Distance 1->4:", wg.distance(1, 4))
    print("Shortest path 2->4:", wg.shortest_path(2, 4))
    print("Distance 2->4:", wg.distance(2, 4))

if __name__ == "__main__":
    test_graph()

1 -> [(2, 3), (3, 1)]
2 -> [(3, 7), (4, 5)]
3 -> [(4, 2)]
4 -> []
Nodes: [1, 2, 3, 4]
Edges: [(1, 2, 3), (1, 3, 1), (2, 3, 7), (2, 4, 5), (3, 4, 2)]
Shortest path 1->4: [1, 3, 4]
Distance 1->4: 3
Shortest path 2->4: [2, 4]
Distance 2->4: 5


In [2]:

import unittest
from Graphs import MyGraph

class TestMyGraph(unittest.TestCase):
    def setUp(self):
        self.g = MyGraph({
            1: [(2, 3), (3, 1)],
            2: [(3, 7), (4, 5)],
            3: [(4, 2)],
            4: []
        })

    def test_nodes_and_edges(self):
        self.assertEqual(set(self.g.get_nodes()), {1, 2, 3, 4})
        self.assertIn((1, 2, 3), self.g.get_edges())
        self.assertEqual(self.g.size(), (4, 5))

    def test_add_vertex_and_edge(self):
        self.g.add_vertex(5)
        self.assertIn(5, self.g.get_nodes())
        self.g.add_edge(5, 1, 10)
        self.assertIn((5, 1, 10), self.g.get_edges())

    def test_successors_predecessors(self):
        self.assertEqual(set(self.g.get_successors(1)), {2, 3})
        self.assertEqual(set(self.g.get_predecessors(3)), {1, 2})
        self.assertEqual(set(self.g.get_adjacents(3)), {1, 2, 4})

    def test_degrees(self):
        self.assertEqual(self.g.out_degree(2), 2)
        self.assertEqual(self.g.in_degree(3), 2)
        self.assertEqual(self.g.degree(3), 3)

    def test_shortest_path_and_distance(self):
        self.assertEqual(self.g.shortest_path(1, 4), [1, 3, 4])
        self.assertEqual(self.g.distance(1, 4), 3)
        self.assertEqual(self.g.shortest_path(1, 1), [1])
        self.assertIsNone(self.g.shortest_path(4, 1))

    def test_reachability(self):
        self.assertEqual(set(self.g.reachable_bfs(1)), {2, 3, 4})
        self.assertEqual(set(self.g.reachable_dfs(1)), {2, 3, 4})
        dists = dict(self.g.reachable_with_dist(1))
        self.assertEqual(dists[4], 2)

    def test_cycles(self):
        self.assertFalse(self.g.has_cycle())
        self.g.add_edge(4, 1, 1)
        self.assertTrue(self.g.has_cycle())

    def test_empty_graph(self):
        g = MyGraph({})
        self.assertEqual(g.get_nodes(), [])
        self.assertEqual(g.get_edges(), [])
        self.assertEqual(g.size(), (0, 0))
        self.assertFalse(g.has_cycle())

    def test_self_loop(self):
        g = MyGraph({1: [(1, 1)]})
        self.assertTrue(g.has_cycle())
        self.assertEqual(g.get_successors(1), [1])
        self.assertEqual(g.get_predecessors(1), [1])
        self.assertEqual(g.get_adjacents(1), [1])
        self.assertEqual(g.degree(1), 2)

    def test_disconnected_graph(self):
        g = MyGraph({1: [], 2: [], 3: []})
        self.assertEqual(g.get_edges(), [])
        self.assertFalse(g.has_cycle())
        for node in g.get_nodes():
            self.assertEqual(g.out_degree(node), 0)
            self.assertEqual(g.in_degree(node), 0)
            self.assertEqual(g.degree(node), 0)

suite = unittest.TestLoader().loadTestsFromTestCase(TestMyGraph)
unittest.TextTestRunner(verbosity=2).run(suite)

test_add_vertex_and_edge (__main__.TestMyGraph.test_add_vertex_and_edge) ... ok
test_cycles (__main__.TestMyGraph.test_cycles) ... ok
test_degrees (__main__.TestMyGraph.test_degrees) ... ok
test_disconnected_graph (__main__.TestMyGraph.test_disconnected_graph) ... ok
test_empty_graph (__main__.TestMyGraph.test_empty_graph) ... ok
test_nodes_and_edges (__main__.TestMyGraph.test_nodes_and_edges) ... ok
test_reachability (__main__.TestMyGraph.test_reachability) ... ok
test_self_loop (__main__.TestMyGraph.test_self_loop) ... ok
test_shortest_path_and_distance (__main__.TestMyGraph.test_shortest_path_and_distance) ... ok
test_successors_predecessors (__main__.TestMyGraph.test_successors_predecessors) ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.046s

OK


<unittest.runner.TextTestResult run=10 errors=0 failures=0>